## Data Science Jobs in India - Naukri

<strong><a href="https://www.naukri.com/">Naukri.com</a></strong> is an Indian employment website operating in India and Middle East. Naukri.com was founded in March 1997. Naukri was ranked No.1 by 9 independent sources, placing it way ahead of competition.Google Trends names Naukri “the most preferred job search destination in India”.

In this project, I web scraped first 12,000 results obtained for <strong>'data scientist'</strong> job role in <strong>India</strong>

In [1]:
# pip install selenium

In [2]:
# import libraries

import pandas as pd
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

if you are using Chrome, use the code <br>
```
driver = webdriver.Chrome(executable_path='/path/chromedriver')
```
- To download chromedriver, <a href="https://chromedriver.storage.googleapis.com/index.html?path=101.0.4951.15/">click here</a>

In [3]:
# webdriver for Firefox - geckodriver
driver = webdriver.Firefox(executable_path='/home/anandhuh/Documents/geckodriver')

# sticky timeout to implicitly wait for an element to be found
driver.implicitly_wait(20) 

# open url in the browser
driver.get('https://www.naukri.com/')

In [4]:
# enter job role - data scientist
role=driver.find_element_by_xpath('/html/body/div/div[2]/div[3]/div/div/div[1]/div/div/div/input')
role.send_keys('data scientist')

# location - India
location=driver.find_element_by_xpath('/html/body/div/div[2]/div[3]/div/div/div[3]/div/div/div/input')
location.send_keys('india')

# click search
driver.find_element_by_class_name('qsbSubmit').click()

In [5]:
# new dataframe fwith required columns
df=pd.DataFrame(columns=['Job_Role','Company','Location','Job Experience','Skills/Description'])
df

,Job_Role,Company,Location,Job Experience,Skills/Description


In [6]:
# helper function

# Function to convert  a list to string
def listToString(s): 
    
    # initialize an empty string
    str1 = ", " 
    
    # return string  
    return (str1.join(s))

In [7]:
%%time

# page number
page = 1

condition = True

while condition:
    
    try:
        
        # 20 results per page
        for i in range(1,21):

            role = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/a'.format(i)).text
            company = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/div/a[1]'.format(i)).text
            location = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div[1]/ul/li[3]/span[1]'.format(i)).text
            experience = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/div[1]/div/ul/li[1]/span'.format(i)).text.replace(' Yrs','')
            skill_lst = driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[2]/article[{}]/ul'.format(i)).text.split('\n')
            skill = listToString(skill_lst)
            
            # append collected information to the dataframe df
            df.loc[len(df.index)] = [role,company,location,experience,skill]
        
        page += 1
        
        # click next
        driver.find_element_by_xpath('/html/body/div/div[3]/div[2]/section[2]/div[3]/div/a[2]').click()

        
    except NoSuchElementException:
        
        # refresh page in case of exception
        driver.refresh() 
        
    
    # total 12700+ results showed, 20 results per page => 600+ pages
    
    # stop after 600 pages
    if page > 600:
        condition = False

CPU times: user 2min 22s, sys: 4.73 s, total: 2min 27s
Wall time: 24min 2s


In [8]:
# shape of the dataframe
df.shape

(12000, 5)

12,000 results are scraped into a dataframe

In [9]:
# first five rows
df.head()

,Job_Role,Company,Location,Job Experience,Skills/Description
0,Data Scientist,Novo Nordisk India,Bangalore/Bengaluru,3-8,"Process automation, Telecom, Data analysis, da..."
1,Senior/ Lead Data Scientist - Market Mix Model...,BLEND360 INDIA PRIVATE LIMITED,Hyderabad/Secunderabad,3-8,"Data Scientist, Market Mix Modelling, Pyspark,..."
2,Data Analyst / Data Scientist,JK Technosoft,"Noida, Hyderabad/Secunderabad, Bangalore/Benga...",5-7,"SAP, SQL Server, Oracle, analytical, IT Skills..."
3,Sr. Data Scientist,Thomson Reuters,Bangalore/Bengaluru,3-6,"Computer science, Customer acquisition, Busine..."
4,Senior Data Scientist (Computational Geometry),HERE Technologies,Bangalore/Bengaluru,4-7,"data science, computational geometry, machine ..."


In [10]:
# last five rows
df.tail()

,Job_Role,Company,Location,Job Experience,Skills/Description
11995,NLP Python - Developer,Anicalls,Hyderabad/Secunderabad,5-9,"AWS, Translation, development testing, Neural ..."
11996,Python Developer,Carta Healthcare,"Mumbai, Navi Mumbai, Mumbai (All Areas)",4-5,"python, postgresql, flask, IT Skills, Software..."
11997,Python Developer,Coditas Technologies Pvt Ltd,Pune,3-6,"IT Skills, Python, Javascript, Backend, Front ..."
11998,Python/ Django Developer,StarClinch,New Delhi,2-5,"Computer science, Front end, HP data protector..."
11999,Python Developer,Seven Consultancy (HR Solution),Mumbai,3-8,"MIN, Data management, Consultancy, MVC, Manage..."


In [11]:
# save as excel file
df.to_excel('naukri_data_science_jobs_india.xlsx',index=False)